In [2]:
#Step 1) Write a Python script that fetches holiday data and saves it as a csv file named "holidays.csv". To obtain the data, you can either parse the API or screen scrape the Nager.Date site.
import csv

def fetch_holiday_data(country_code="US", year=2023):
    url = f"https://date.nager.at/api/v3/publicholidays/{year}/{country_code}"
    response = requests.get(url)
    return response.json()

def save_to_csv(data):
    with open("holidays.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "date", "type", "global"])
        for holiday in data:
            writer.writerow([holiday["name"], holiday["date"], holiday.get("type", ""), holiday["global"]])

if __name__ == "__main__":
    data = fetch_holiday_data()
    save_to_csv(data)


In [3]:
#Step 2 and 3- Using the csv file from 1), write a Python function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates,
import csv
from datetime import datetime

def get_next_10_holidays(global_filter=None):
    with open("holidays.csv", "r") as f:
        reader = csv.DictReader(f)
        holidays_list = [row for row in reader]

    today = datetime.today().date()
    upcoming_holidays = [h for h in holidays_list if datetime.strptime(h["date"], '%Y-%m-%d').date() >= today]
    
    if global_filter is not None:
        upcoming_holidays = [h for h in upcoming_holidays if h["global"] == str(global_filter)]
    
    return upcoming_holidays[:10]

# Test
print(get_next_10_holidays())


[{'name': 'Columbus Day', 'date': '2023-10-09', 'type': '', 'global': 'False'}, {'name': 'Veterans Day', 'date': '2023-11-10', 'type': '', 'global': 'True'}, {'name': 'Thanksgiving Day', 'date': '2023-11-23', 'type': '', 'global': 'True'}, {'name': 'Christmas Day', 'date': '2023-12-25', 'type': '', 'global': 'True'}]


In [4]:
#Step 4 - Add an additional keyword argument "comparison_country" to the function from 3) such that it only returns holidays that share the same date of the comparison country.

def fetch_holiday_data(country_code="US", year=2023):
    url = f"https://date.nager.at/api/v3/publicholidays/{year}/{country_code}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for country: {country_code}")
        return []
    try:
        return response.json()
    except JSONDecodeError:
        print(f"Failed to decode JSON for country: {country_code}")
        return []

def most_common_countries(comparison_country="US"):
    current_year = datetime.now().year
    comparison_holidays = fetch_holiday_data(country_code=comparison_country, year=current_year)
    comparison_dates = {h["date"] for h in comparison_holidays}

    common_count = {}
    countries = ["AT", "BE", "BG", "BR", "CA", "CH", "CN", "CO", "CZ", "DE", "DK", "EE", "ES", "FI", "FR", "GB", "GR", "HR", "HU", "IE", "IN", "IT", "JP", "KR", "MX", "NL", "NO", "NZ", "PL", "PT", "RO", "RU", "SE", "SG", "SI", "SK", "ZA", "US"]
    for country in countries:
        country_holidays = fetch_holiday_data(country_code=country, year=current_year)
        common_dates = comparison_dates.intersection({h["date"] for h in country_holidays if "date" in h})
        common_count[country] = len(common_dates)

    sorted_countries = sorted(common_count.items(), key=lambda x: x[1], reverse=True)[:10]
    result = [{"country": k, "common_dates": v} for k, v in sorted_countries]
    return result

# Test
print(most_common_countries())


Failed to fetch data for country: IN
[{'country': 'US', 'common_dates': 12}, {'country': 'CA', 'common_dates': 5}, {'country': 'CH', 'common_dates': 4}, {'country': 'ES', 'common_dates': 4}, {'country': 'GB', 'common_dates': 4}, {'country': 'BE', 'common_dates': 3}, {'country': 'BR', 'common_dates': 3}, {'country': 'CO', 'common_dates': 3}, {'country': 'DE', 'common_dates': 3}, {'country': 'DK', 'common_dates': 3}]


In [5]:
#Step 5 - Write a Python function that returns a list of dictionaries of the top 10 countries ranked by the number of upcoming holiday dates (in the next 364 days) they have in common with a given comparison country

def fetch_holiday_data(country_code="US", year=2023):
    url = f"https://date.nager.at/api/v3/publicholidays/{year}/{country_code}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for country: {country_code}")
        return []
    try:
        return response.json()
    except JSONDecodeError:
        print(f"Failed to decode JSON for country: {country_code}")
        return []

def most_common_countries(comparison_country="US"):
    current_year = datetime.now().year
    comparison_holidays = fetch_holiday_data(country_code=comparison_country, year=current_year)
    comparison_dates = {h["date"] for h in comparison_holidays}

    common_count = {}
    countries = ["AT", "BE", "BG", "BR", "CA", "CH", "CN", "CO", "CZ", "DE", "DK", "EE", "ES", "FI", "FR", "GB", "GR", "HR", "HU", "IE", "IN", "IT", "JP", "KR", "MX", "NL", "NO", "NZ", "PL", "PT", "RO", "RU", "SE", "SG", "SI", "SK", "ZA", "US"]
    for country in countries:
        country_holidays = fetch_holiday_data(country_code=country, year=current_year)
        common_dates = comparison_dates.intersection({h["date"] for h in country_holidays if "date" in h})
        common_count[country] = len(common_dates)

    sorted_countries = sorted(common_count.items(), key=lambda x: x[1], reverse=True)[:10]
    result = [{"country": k, "common_dates": v} for k, v in sorted_countries]
    return result

# Test
print(most_common_countries())


Failed to fetch data for country: IN
[{'country': 'US', 'common_dates': 12}, {'country': 'CA', 'common_dates': 5}, {'country': 'CH', 'common_dates': 4}, {'country': 'ES', 'common_dates': 4}, {'country': 'GB', 'common_dates': 4}, {'country': 'BE', 'common_dates': 3}, {'country': 'BR', 'common_dates': 3}, {'country': 'CO', 'common_dates': 3}, {'country': 'DE', 'common_dates': 3}, {'country': 'DK', 'common_dates': 3}]
